Atalov S.

Introduction to Machine Learning and Artificial Intelligence

# Data Preparation Pipeline
---


<img src="https://akujaakko.com/images/pipeline.gif">

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

## 1. Data Preparation

In [2]:
titanic_train = 'https://raw.githubusercontent.com/lobachevksy/teaching/main/titanic/train.csv'
df = pd.read_csv(titanic_train)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
df

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,0,0,0,1
1,1,1,38.000000,1,0,71.2833,1,1,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,1
3,1,1,35.000000,1,0,53.1000,1,0,0,1
4,0,3,35.000000,0,0,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,0,0,0,1
887,1,1,19.000000,0,0,30.0000,1,0,0,1
888,0,3,29.699118,1,2,23.4500,1,0,0,1
889,1,1,26.000000,0,0,30.0000,0,1,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# Выделяем столбцы для удаления, численные и категориальные

drop_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']

### TO DO
numerical_cols = [ "Pclass", "Age", "SibSp", "Parch", "Fare" ]
categorical_cols = [ "Sex", "Embarked" ]

In [5]:
df = df.drop(columns = drop_cols)

In [6]:
# ФУНКЦИИ для автотматизации подготовки данных

def fill_missing_values(df: pd.DataFrame, strategy: str ='mean') -> pd.DataFrame:
    """Заполнение пустых значений указанными стратегиями"""
    df = df.copy()
    fill_value = 0
    for column in df.columns:
        if strategy == 'mean':
            fill_value = df[column].mean()
        elif strategy == 'median':
            fill_value = df[column].median()
        elif strategy == 'mode':
            fill_value = df[column].mode()[0] # берем первую моду
            
        ### TO DO
        df[column].fillna( value = fill_value, inplace=True )
    return df

def one_hot_encoder(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """Категорийные данные в численные"""
    df = df.copy()
    # Убираем численные столбцы - их не трогаем
    df_num = df.drop(columns = columns)
    
    # В датафрейме оставляем только категорийные столбцы
    df = df[columns].copy()
    
    # Новый датафрейм который будем возвращать
    df_prep = pd.DataFrame()
    
    for column in df.columns:
        # Уникальные значения столбца
        unique_values = df[column].unique()
        # Если уникальных значений меньше 2, то просто replace
        if len(unique_values) <= 2:
            df_prep[column] = df[column]
            for i in range(len(unique_values)):
                df_prep[column] = df_prep[column].replace({unique_values[i]: i})
        else: # Иначе get_dummies
            dummies = pd.get_dummies( df[column], prefix=column )
            df_prep = pd.concat([df_prep, dummies], axis=1)
    
    # Соединяем численные и категорийные столбцы
    return pd.concat([df_num, df_prep], axis=1)

In [7]:
df[numerical_cols] = fill_missing_values( df[numerical_cols] )
df[categorical_cols] = fill_missing_values( df[categorical_cols], strategy='mode' )

In [8]:
df = one_hot_encoder( df, categorical_cols )

In [9]:
X = df.copy()

y = X.pop('Survived')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## 2. Modeling

In [11]:
model = RandomForestClassifier(
    max_depth = 8,
    min_samples_split = 5,
    n_estimators = 120
)

model.fit(X_train, y_train)
model.score(X_train, y_train)
model.score(X_test, y_test)

0.8268156424581006

In [12]:
import pickle

In [13]:
# сохранить оцененную модель (с гиперпараметрами) на диск
filename = 'titanic_model.pkl'
pickle.dump(model, open(filename, 'wb'))

## 3. Making a predictions

Напишите функции для автоматической подготовки данных и предсказания на основе данных

In [14]:
titanic_train = 'https://raw.githubusercontent.com/lobachevksy/teaching/main/titanic/train.csv'
df_test = pd.read_csv(titanic_train)


In [15]:
def make_predictions(X: pd.DataFrame, model_name: str) -> np.array:
    # Вызываются все функции для подготовки и предсказанияX = X.drop(columns = drop_cols)
    
    X = X.drop(columns = drop_cols)
    X = X.drop(columns=['Survived'])
    X[numerical_cols] = fill_missing_values( X[numerical_cols] )
    X[categorical_cols] = fill_missing_values( X[categorical_cols], strategy='mode' )
    
    X = one_hot_encoder( X, categorical_cols )
    
    return predict(X, model_name)

def predict(data, model_name):
    # Load the model from disk
    with open(model_name, 'rb') as file:
        model = pickle.load(file)
        
    y_pred = model.predict(data)
    return y_pred

In [16]:
make_predictions(df_test, "titanic_model.pkl")

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,